In [11]:
from dotenv import load_dotenv
import os
import requests
import alpaca_trade_api as tradeapi
import pandas as pd
import numpy as np

In [2]:
load_dotenv('api.env')

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [4]:
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [5]:
alpaca = tradeapi.REST(
    alpaca_api_key,alpaca_secret_key,
    api_version="v2")

In [6]:
symbol='SPY'
timeframe = '1H'
start_date = pd.Timestamp("2013-01-01", tz="America/New_York").isoformat()

In [18]:
historical_data = alpaca.get_bars(symbol, timeframe, start=start_date).df


In [19]:
historical_data

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.44,209.24,209.30,30375,82,209.322606
2015-12-01 10:00:00+00:00,209.30,209.36,209.07,209.36,54490,118,209.214253
2015-12-01 11:00:00+00:00,209.37,209.65,209.37,209.44,44404,109,209.526441
2015-12-01 12:00:00+00:00,209.45,209.59,209.33,209.51,111106,234,209.460429
2015-12-01 13:00:00+00:00,209.52,209.67,209.27,209.32,426233,645,209.429479
...,...,...,...,...,...,...,...
2023-03-22 19:00:00+00:00,398.56,399.89,392.07,392.14,36357678,311668,396.150776
2023-03-22 20:00:00+00:00,392.16,392.99,391.87,392.35,11913426,13993,393.085316
2023-03-22 21:00:00+00:00,392.41,392.65,392.22,392.40,135961,1449,392.439471


**VWAP Trading Signal**

We decide to buy/sell at points where the market price crosses the vwap.

In [16]:
historical_data['Predicted']=np.where(historical_data['vwap']>=historical_data['close'], 1,0)
historical_data['entry/exit']=historical_data['Predicted'].diff()
historical_data

,open,high,low,close,volume,trade_count,vwap,Predicted,entry/exit
timestamp,,,,,,,,,
2015-12-01 09:00:00+00:00,209.43,209.44,209.24,209.30,30375,82,209.322606,1,NaN
2015-12-01 10:00:00+00:00,209.30,209.36,209.07,209.36,54490,118,209.214253,0,-1.0
2015-12-01 11:00:00+00:00,209.37,209.65,209.37,209.44,44404,109,209.526441,1,1.0
2015-12-01 12:00:00+00:00,209.45,209.59,209.33,209.51,111106,234,209.460429,0,-1.0
2015-12-01 13:00:00+00:00,209.52,209.67,209.27,209.32,426233,645,209.429479,1,1.0
...,...,...,...,...,...,...,...,...,...
2023-03-22 19:00:00+00:00,398.56,399.89,392.07,392.14,36357678,311668,396.150776,1,0.0
2023-03-22 20:00:00+00:00,392.16,392.99,391.87,392.35,11913426,13993,393.085316,1,0.0
2023-03-22 21:00:00+00:00,392.41,392.65,392.22,392.40,135961,1449,392.439471,1,0.0
